In [7]:
import json 
from typing import Dict, Any

extracted_metrics = json.loads(open(r'C:\Users\User\Projects\FYP2\results\centralized\logs\metrics_output\metrics.json').read())

In [8]:
extracted_metrics

{'best_model_path': 'C:\\Users\\User\\Projects\\FYP2\\results\\centralized\\checkpoints\\pneumonia_model_00_0.750.ckpt',
 'best_model_score': 0.75,
 'current_epoch': 6,
 'global_step': 12,
 'state': None,
 'model_summary': {'model_name': 'ResNetWithCustomHead',
  'backbone': 'ResNet50',
  'backbone_weights': 'ResNet50_Weights.IMAGENET1K_V2',
  'num_classes': 1,
  'dropout_rate': 0.3,
  'fine_tune_layers_count': 5,
  'total_parameters': 24049089,
  'trainable_parameters': 541057,
  'backbone_parameters': 23508032,
  'backbone_trainable_parameters': 0,
  'head_parameters': 541057,
  'trainable_ratio': 0.022498024769254252,
  'input_size': [256, 256],
  'architecture': 'Sequential(\n  (0): AdaptiveAvgPool2d(output_size=(1, 1))\n  (1): Flatten(start_dim=1, end_dim=-1)\n  (2): Linear(in_features=2048, out_features=256, bias=True)\n  (3): ReLU(inplace=True)\n  (4): Dropout(p=0.3, inplace=False)\n  (5): Linear(in_features=256, out_features=64, bias=True)\n  (6): ReLU(inplace=True)\n  (7): Dro

In [9]:
import json

def filter_list_of_dicts(data:list[dict[str, Any]], fields:list[str]):
  """
  Filters a list of dictionaries to include only specified keys.

  Args:
    data: A list of dictionaries.
    fields: A list of keys to keep in each dictionary.

  Returns:
    A new list of dictionaries with only the specified keys.
  """
  metrics = {}
  for metric in data:
    for k,v in metric.items():
      if k in fields:
        metrics.update({k: v})
    
  return metrics



# The fields you want to keep
fields_to_keep = [
 'train_loss', 'train_acc', 'train_f1', 'val_loss', 
    'val_acc', 'val_precision', 'val_recall', 'val_f1', 'val_auroc'
]

metrics_history = extracted_metrics['metrics_history']
# Call the function with your data and fields
filtered_data = filter_list_of_dicts(metrics_history, fields_to_keep)

# Pretty print the result
print(json.dumps(filtered_data, indent=2))

{
  "val_loss": 0.8898273706436157,
  "val_acc": 0.800000011920929,
  "val_precision": 0.6666666865348816,
  "val_recall": 0.5,
  "val_f1": 0.5714285969734192,
  "val_auroc": 0.75,
  "train_loss": 0.9005182385444641,
  "train_acc": 0.7166666388511658,
  "train_f1": 0.5641025900840759
}


In [10]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from flwr_datasets.partitioner import Partitioner
from federated_pneumonia_detection.src.entities.custom_image_dataset import (
    CustomImageDataset,
)
from federated_pneumonia_detection.src.utils.image_transforms import TransformBuilder
from federated_pneumonia_detection.src.utils.config_loader import ConfigLoader


class CustomPartitioner(Partitioner):
    def __init__(self, base_dataset, num_partitions: int):
        super().__init__()
        self._base_dataset = base_dataset  # Your full PyTorch Dataset instance
        self._num_partitions = num_partitions

        total_size = len(base_dataset)
        indices = np.random.permutation(total_size)  # Shuffle
        self.partition_indices = np.array_split(
            indices, num_partitions
        )  # Evenly split indices

    @property
    def num_partitions(self):
        return self._num_partitions

    def load_partition(self, partition_id: int) -> CustomImageDataset:
        assert 0 <= partition_id < self._num_partitions
        partition_idx = self.partition_indices[partition_id]
        # Return a Subset of your base Dataset
        return self._base_dataset.iloc[partition_idx].reset_index(drop=True)

2025-11-01 14:27:32 - datasets - INFO - PyTorch version 2.8.0+cu128 available. - config.py - 54


In [11]:
dataset = pd.read_csv(r"C:\Users\User\Projects\FYP2\Training_Sample_5pct\stage2_train_metadata.csv")

In [12]:
dataset['filename'] = dataset.apply(lambda x: x['patientId']+'.png', axis=1)

partioner = CustomPartitioner(dataset, 10)

partition = partioner.load_partition(0)

for nums in range(10):
    partition = partioner.load_partition(nums)
    print('-'*500)
    print(partition)


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                                patientId      x      y  width  height  \
0    f7bf7dd9-76af-4fbe-bfc7-a524691da76b    NaN    NaN    NaN     NaN   
1    bccb1c1f-d429-4ed4-be42-808a75b635f9  655.0  473.0  177.0   270.0   
2    370e4257-86fb-40b7-a4df-b8a3a7b25226  181.0  157.0  282.0   603.0   
3    4f9a7303-4d5d-45fc-bcc4-d3f50f03464e    NaN    NaN    NaN     NaN   
4    46e6b4be-e2aa-4b5a-8917-9c0846ac0430  295.0  324.0  201.0   298.0   
..                                    ...    ...    ...

In [13]:
print(result)

20


In [1]:
from federated_pneumonia_detection.src.utils.config_loader import ConfigLoader
config_loader = ConfigLoader(
        config_dir=r"federated_pneumonia_detection\config",
        
    )
config = config_loader.load_config()
    

2025-11-01 14:31:34 - federated_pneumonia_detection.src.utils.config_loader - INFO - Configuration loaded from federated_pneumonia_detection/config/default_config.yaml (absolute: C:\Users\User\Projects\FYP2\federated_pneumonia_detection\config\default_config.yaml) - config_loader.py - 68
2025-11-01 14:31:34 - federated_pneumonia_detection.src.utils.config_loader - INFO - [ConfigLoader] Loaded from YAML - epochs=10, batch_size=32, learning_rate=0.001 - config_loader.py - 73


In [2]:
config['experiment']

{'adaptive_histogram': False,
 'augmentation_strength': 1.0,
 'batch_size': 32,
 'clip_limit': 2.0,
 'color_mode': 'RGB',
 'contrast_stretch': True,
 'device': 'auto',
 'dropout_rate': 0.3,
 'early_stopping_patience': 5,
 'edge_enhancement': False,
 'edge_strength': 1.0,
 'epochs': 10,
 'fine_tune_layers_count': 5,
 'freeze_backbone': True,
 'learning_rate': 0.001,
 'local_epochs': 4,
 'lower_percentile': 5.0,
 'max-epochs': 1,
 'min_lr': 1e-07,
 'monitor_metric': 'val_loss',
 'num-server-rounds': 20,
 'num_classes': 1,
 'num_clients': 2,
 'num_rounds': 3,
 'num_workers': 0,
 'options': {'num-supernodes': 4},
 'persistent_workers': False,
 'pin_memory': True,
 'prefetch_factor': 2,
 'reduce_lr_factor': 0.5,
 'reduce_lr_patience': 3,
 'upper_percentile': 95.0,
 'use_custom_preprocessing': False,
 'use_imagenet_norm': True,
 'validate_images_on_init': True,
 'weight_decay': 0.0001}

In [1]:
from federated_pneumonia_detection.src.control.federated_new_version.toml_adjustment import update_flwr_config
from federated_pneumonia_detection.src.control.federated_new_version.server_app import read_configs_to_toml
from federated_pneumonia_detection.config.config_manager import ConfigManager

config_manager = ConfigManager(config_path=r"federated_pneumonia_detection\config\default_config.yaml")
result = config_manager.get('experiment.num-server-rounds')
print(result)




c:\Users\User\Projects\FYP2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-01 14:39:38,903	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


20


In [2]:
config_manager.set('experiment.num-server-rounds', 20)

print(config_manager.get('experiment.num-server-rounds'))
config_manager.set('experiment.options.num-supernodes', 20)
print(config_manager.get('experiment.options.num-supernodes'))
config_manager.set('experiment.max-epochs', 20)

config_manager.save()

2025-11-01 14:39:39 - federated_pneumonia_detection.config.config_manager - INFO - Saving configuration to federated_pneumonia_detection\config\default_config.yaml - config_manager.py - 294


20
20


In [3]:
from federated_pneumonia_detection.src.control.federated_new_version.server_app import read_configs_to_toml
import tomli

result = read_configs_to_toml()


with open(
    r"federated_pneumonia_detection\src\control\federated_new_version\pyproject.toml",
    "rb",
) as f:
    print(tomli.load(f))

{'num-server-rounds': 20, 'max-epochs': 20, 'options.num-supernodes': 20}
{'build-system': {'requires': ['hatchling'], 'build-backend': 'hatchling.build'}, 'project': {'name': 'pytorchlightning_example', 'version': '1.0.0', 'description': 'Federated Learning with PyTorch Lightning and Flower (Quickstart Example)', 'license': 'Apache-2.0', 'dependencies': ['flwr[simulation]>=1.22.0', 'flwr-datasets[vision]>=0.5.0', 'pytorch-lightning==2.5.5', 'torch==2.8.0', 'torchvision==0.23.0']}, 'tool': {'hatch': {'build': {'targets': {'wheel': {'packages': ['.']}}}}, 'flwr': {'app': {'publisher': 'flwrlabs', 'components': {'serverapp': 'server_app:app', 'clientapp': 'client_app:app'}, 'config': {'num-server-rounds': 5, 'max-epochs': 1, 'num_server_rounds': 20, 'max_epochs': 20}}, 'federations': {'default': 'local-simulation', 'local-simulation': {'options.num-supernodes': 4, 'options_num_supernodes': 20, 'options': {'num-supernodes': 4}}}}}}
